# Quickstart: Biological Countercurvature of Spacetime

This notebook demonstrates the core countercurvature framework:

1. **Information field I(s)**: Represents biological information processing (neural control, growth programs)
2. **Countercurvature metric g_eff(s)**: Encodes how information reshapes effective geometry
3. **Geodesic deviation D̂_geo**: Measures how far information-driven curvature departs from gravity-selected profiles

The framework treats a Cosserat rod in gravity as an "analog spacetime" where information fields modify the effective metric, implementing the hypothesis that biological information processing acts as a local countercurvature to gravitational geometry.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from spinalmodes.countercurvature import (
    InfoField1D,
    CounterCurvatureParams,
    compute_countercurvature_metric,
    geodesic_curvature_deviation,
    make_uniform_grid,
)
from spinalmodes.countercurvature.coupling import (
    compute_rest_curvature,
)
from spinalmodes import solve_beam_static

# Set up problem
length = 0.4  # meters (spine-like structure)
n_points = 100
s = make_uniform_grid(length, n_points)

# Create information field I(s) with peaks in lumbar and cervical regions
s_norm = s / length
lumbar = 0.7 * np.exp(-((s_norm - 0.25) ** 2) / (2 * 0.1**2))
cervical = 0.5 * np.exp(-((s_norm - 0.8) ** 2) / (2 * 0.08**2))
I = lumbar + cervical + 0.3
info_field = InfoField1D.from_array(s, I)

print(f"✅ Created information field I(s) with {len(s)} points")
print(f"   I(s) range: [{I.min():.3f}, {I.max():.3f}]")


In [ ]:
# Solve beam equilibrium for passive (gravity-only) and info-driven cases
kappa_gen = np.zeros_like(s)  # Baseline curvature
E0 = 1e9  # Pa
I_moment = 1e-8  # m^4
gravity_load = 100.0  # N/m

# Passive case (no info coupling)
params_passive = CounterCurvatureParams(chi_kappa=0.0, chi_E=0.0, chi_M=0.0)
kappa_rest_passive = compute_rest_curvature(info_field, params_passive, kappa_gen)
E_passive = np.full_like(s, E0)
M_passive = np.zeros_like(s)
_, kappa_passive = solve_beam_static(
    s, kappa_rest_passive, E_passive, M_passive,
    I_moment=I_moment, distributed_load=gravity_load
)

# Info-driven case (with coupling)
chi_kappa = 0.03  # Information-to-curvature coupling strength
params_info = CounterCurvatureParams(
    chi_kappa=chi_kappa, chi_E=0.1, chi_M=0.0, scale_length=length
)
kappa_rest_info = compute_rest_curvature(info_field, params_info, kappa_gen)
E_info = E0 * (1.0 + 0.1 * info_field.I)  # Simplified stiffness modulation
M_info = np.zeros_like(s)  # No active moments for this demo
_, kappa_info = solve_beam_static(
    s, kappa_rest_info, E_info, M_info,
    I_moment=I_moment, distributed_load=gravity_load
)

print(f"✅ Solved beam equilibrium for passive and info-driven cases")
print(f"   Passive curvature range: [{kappa_passive.min():.3f}, {kappa_passive.max():.3f}] 1/m")
print(f"   Info-driven curvature range: [{kappa_info.min():.3f}, {kappa_info.max():.3f}] 1/m")


In [ ]:
# Compute countercurvature metric and geodesic deviation
g_eff = compute_countercurvature_metric(info_field, beta1=1.0, beta2=0.5)
geo_metrics = geodesic_curvature_deviation(
    s, kappa_passive, kappa_info, g_eff
)

D_geo = geo_metrics["D_geo"]
D_geo_norm = geo_metrics["D_geo_norm"]

print(f"✅ Computed countercurvature metrics")
print(f"   D_geo = {D_geo:.6f} 1/m")
print(f"   D̂_geo = {D_geo_norm:.4f} (normalized)")
print(f"   Base energy = {geo_metrics['base_energy']:.6e}")
print()
print(f"Interpretation: D̂_geo = {D_geo_norm:.4f} indicates")
if D_geo_norm < 0.1:
    print("   → Gravity-dominated regime (information has minimal effect)")
elif D_geo_norm < 0.3:
    print("   → Cooperative regime (information reshapes but doesn't override gravity)")
else:
    print("   → Information-dominated regime (effective geometry strongly warped)")


In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Panel A: Information field I(s)
ax = axes[0, 0]
ax.plot(s / length, I, "g-", linewidth=2, label="I(s)")
ax_twin = ax.twinx()
ax_twin.plot(s / length, info_field.dIds, "orange", linewidth=1.5, linestyle="--", label="∂I/∂s")
ax.set_xlabel("Arc-length s/L")
ax.set_ylabel("Information density I(s)", color="g")
ax_twin.set_ylabel("Gradient ∂I/∂s", color="orange")
ax.set_title("(A) Information Field")
ax.legend(loc="upper left")
ax_twin.legend(loc="upper right")
ax.grid(alpha=0.3)

# Panel B: Countercurvature metric g_eff(s)
ax = axes[0, 1]
ax.plot(s / length, g_eff, "purple", linewidth=2, label="g_eff(s)")
ax.axhline(1.0, color="k", linestyle=":", alpha=0.5, label="Flat metric")
ax.set_xlabel("Arc-length s/L")
ax.set_ylabel("Countercurvature metric g_eff(s)")
ax.set_title("(B) Biological Countercurvature Metric")
ax.legend()
ax.grid(alpha=0.3)
ax.set_yscale("log")

# Panel C: Curvature profiles
ax = axes[1, 0]
ax.plot(s / length, kappa_passive, "b-", linewidth=2, label="κ_passive (gravity-only)")
ax.plot(s / length, kappa_info, "r-", linewidth=2, label="κ_info (info-coupled)")
ax.axhline(0, color="k", linestyle=":", alpha=0.3)
ax.set_xlabel("Arc-length s/L")
ax.set_ylabel("Curvature κ (1/m)")
ax.set_title("(C) Curvature Profiles")
ax.legend()
ax.grid(alpha=0.3)

# Panel D: Metrics summary
ax = axes[1, 1]
ax.axis("off")
regime = "Gravity-dominated" if D_geo_norm < 0.1 else "Cooperative" if D_geo_norm < 0.3 else "Information-dominated"
ax.text(0.1, 0.8, "Geodesic Deviation Metrics", fontsize=14, fontweight="bold")
ax.text(0.1, 0.6, f"D_geo = {D_geo:.6f} 1/m", fontsize=12)
ax.text(0.1, 0.5, f"D̂_geo = {D_geo_norm:.4f}", fontsize=12)
ax.text(0.1, 0.4, f"Base energy = {geo_metrics['base_energy']:.6e}", fontsize=12)
ax.text(0.1, 0.2, f"Coupling: χ_κ = {chi_kappa:.3f}", fontsize=12)
ax.text(0.1, 0.1, f"Regime: {regime}", fontsize=12)

plt.tight_layout()
plt.savefig("quickstart_countercurvature.png", dpi=150, bbox_inches="tight")
plt.show()

print("✅ Saved figure to quickstart_countercurvature.png")


## Next Steps

This quickstart demonstrates the core countercurvature framework. To explore further:

1. **Run full experiments**: See `src/spinalmodes/experiments/countercurvature/`
   - `experiment_phase_diagram.py`: Map regimes in (χ_κ, g) space
   - `experiment_microgravity_adaptation.py`: Test persistence as g → 0
   - `experiment_scoliosis_bifurcation.py`: Explore symmetry breaking

2. **Use PyElastica**: For full 3D Cosserat rod simulations
   ```python
   from spinalmodes.countercurvature import CounterCurvatureRodSystem
   rod_system = CounterCurvatureRodSystem.from_iec(...)
   result = rod_system.run_simulation(final_time=2.0, dt=1e-4)
   ```

3. **Explore scoliosis metrics**: Use `compute_scoliosis_metrics()` for lateral deviation analysis

See the repository documentation for full API reference and examples.
